In [1]:
# import all the E9 stuff
import logging
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# User defined modules
E9path = Path("C:/", "Users", "ken92", "Documents", "Studies", "E5", "simulation", "E9_simulations")
if str(E9path) not in sys.path:
    sys.path.insert(1, str(E9path))
import E9_fn.E9_constants as E9c
import E9_fn.E9_atom as E9a
import E9_fn.E9_cooltrap as E9ct
import E9_fn.polarizabilities_calculation as E9pol
# import E9_fn.datasets.transition_line_data as TLData
from E9_fn import util

# Logging
logpath = '' # '' if not logging to a file
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename = logpath, level = loglevel)

c:\Users\ken92\Documents\Studies\E5\simulation\E9_simulations\.venv\Lib\site-packages\gftool\precision.py:16: UserWarning: No quad precision datatypes available!
Some functions might be less accurate.
  warnings.warn("No quad precision datatypes available!\n"


# Atom stuff

In [2]:
Rb_atom = E9a.Rb87_5_2S1o2_F2
mu_Rb = E9c.mu_B * Rb_atom.gF * 2   # 2 here is mF = 2; sign is V = mu * B
K_atom = E9a.K40_4_2S1o2_F9o2
mu_K = E9c.mu_B * K_atom.gF * 9/2

# Polarizabilities
K_pol = E9pol.alpha_s_K_4S1o2(E9c.lambda_lw)
Rb_pol = E9pol.alpha_s_Rb_5S1o2(E9c.lambda_lw)
KRb_pol_ratio = K_pol / Rb_pol
print("K polarizability is {:.4f} that of Rb polarizability".format(KRb_pol_ratio))

INFO:root:K_4S1o2_3D5o2 transition does not have f_ik data (not E1 allowed?)
INFO:root:K_4S1o2_3D5o2 transition does not have f_ik data (not E1 allowed?)
INFO:root:K_4S1o2_4D5o2 transition does not have f_ik data (not E1 allowed?)
INFO:root:K_4S1o2_4D3o2 transition does not have f_ik data (not E1 allowed?)


K polarizability is 0.8745 that of Rb polarizability


# Loss rates

## Simplified case - harmonic potential, no gravity

In [3]:
# Inputs
wbar_Rb = 2 * np.pi * 200                   # Rb trap frequency in rad * Hz
wbar_K = wbar_Rb * np.sqrt(KRb_pol_ratio)   # K trap frequency in rad * Hz (note the sqrt)
N_Rb = 1e7                                  # Rb atom number
N_K = 5e5                                   # K atom number
T = 3e-6                                    # Temperature in K

In [4]:
n_max_Rb = E9ct.n_peak_har(N_Rb, wbar_Rb, T, Rb_atom.mass)
n_max_K = E9ct.n_peak_har(N_K, wbar_K, T, K_atom.mass)
gamma_BBB_loss_Rb = E9c.G3loss_Rb87_F2mF2 * n_max_Rb**2
gamma_BBF_loss_Rb = E9c.G3loss_Rb87_K40 * n_max_Rb * n_max_K
gamma_BBF_loss_K = E9c.G3loss_Rb87_K40 * n_max_Rb**2
print("Rb loss rate at trap center due to BBB = {:.2e} Hz".format(gamma_BBB_loss_Rb))
print("Rb loss rate at trap center due to BBF = {:.2e} Hz".format(gamma_BBF_loss_Rb))
print("K loss rate at trap center due to BBF = {:.2e} Hz".format(gamma_BBF_loss_K))

Rb loss rate at trap center due to BBB = 7.39e+00 Hz
Rb loss rate at trap center due to BBF = 2.40e-01 Hz
K loss rate at trap center due to BBF = 1.88e+01 Hz


# Trap potentials
Conventions:
* x-axis: EW
    * ODTa is +30 degrees rotated from the x-axis, and ODTb -30 degrees
* y-axis: NS
* z-axis: gravity
* r-axis: radial (of one of the ODT; use ra / rb if it is important to differentiate the two)
* l-axis: optical axis (of one of the ODT)

## ODTs

### Inputs

In [5]:
pwr_ODTa = 3                    # [W] ODT power at atoms
pwr_ODTb = 1                    # [W] ODT power at atoms
w0_ODTa_hori = E9c.w0_ODTa_hori # ODTa horizontal beam waist
w0_ODTa_vert = E9c.w0_ODTa_vert # ODTa vertial beam waist
w0_ODTb_hori = E9c.w0_ODTb_hori # ODTb horizontal beam waist
w0_ODTb_vert = E9c.w0_ODTb_vert # ODTb vertial beam waist

### Find basic ODT parameters from inputs

In [6]:
I_ODTa_max = util.I_from_power(pwr_ODTa, w0_ODTa_hori, w0_ODTa_vert)
I_ODTb_max = util.I_from_power(pwr_ODTb, w0_ODTb_hori, w0_ODTb_vert)

# V_ODTa_FS = E9ct.V0_from_I(E9c.gamma_Rb87_D2, (E9c.nu_Rb87_4_2P3o2 + E9c.nu_Rb87_4_2P1o2)/2,
#                       E9c.f_lw, I_ODTa_max, E9a.Rb87_5_2S1o2_F2.gF, 2)  # only consider D1 and D2 line
V_uK_ODTa_Rb = E9pol.I2uK_from_pol(I_ODTa_max, Rb_pol)
V_uK_ODTa_K = E9pol.I2uK_from_pol(I_ODTa_max, K_pol)
V_uK_ODTb_Rb = E9pol.I2uK_from_pol(I_ODTb_max, Rb_pol)
V_uK_ODTb_K = E9pol.I2uK_from_pol(I_ODTb_max, K_pol)

print("Rayleigh range of the two ODTs:")
print("    ODTa_x = {: 8.4f} mm".format(util.rayleigh_range(w0_ODTa_hori, E9c.lambda_lw) * 1e3))
print("    ODTa_y = {: 8.4f} mm".format(util.rayleigh_range(w0_ODTa_vert, E9c.lambda_lw) * 1e3))
print("    ODTb_x = {: 8.4f} mm".format(util.rayleigh_range(w0_ODTb_hori, E9c.lambda_lw) * 1e3))
print("    ODTb_y = {: 8.4f} mm".format(util.rayleigh_range(w0_ODTb_vert, E9c.lambda_lw) * 1e3))
print("V_ODTa_Rb = {: 10.4f} uK".format(V_uK_ODTa_Rb))
print("V_ODTa_K  = {: 10.4f} uK".format(V_uK_ODTa_K))
print("V_ODTb_Rb = {: 10.4f} uK".format(V_uK_ODTb_Rb))
print("V_ODTb_K  = {: 10.4f} uK".format(V_uK_ODTb_K))

# TODO: find effective trap frequency here

Rayleigh range of the two ODTs:
    ODTa_x =  32.5527 mm
    ODTa_y =   2.0423 mm
    ODTb_x =   7.6798 mm
    ODTb_y =  14.8842 mm
V_ODTa_Rb =  -105.3783 uK
V_ODTa_K  =   -92.1541 uK
V_ODTb_Rb =   -26.7884 uK
V_ODTb_K  =   -23.4266 uK


### Plot the effective harmonic potential
TODO: Probably better to find an explicit expression of curvature

In [ ]:
x_range_trap = np.linspace(-300, 300, 15000) * 1e-6
y_range_trap = np.linspace(-200, 200, 10000) * 1e-6
z_range_trap = np.linspace(-100, 100,  5000) * 1e-6

In [ ]:
theta_ODTa = 30 * (np.pi / 180)
theta_ODTb = -theta_ODTa # by convention (to make use of symmetries)

# x-axis
l_x_range_ODTs = x_range_trap * np.cos(theta_ODTa)
r_x_range_ODTs = x_range_trap * np.sin(theta_ODTa)
l_y_range_ODTs = - y_range_trap * np.sin(theta_ODTa)
r_y_range_ODTs = y_range_trap * np.cos(theta_ODTa)

norm_V_uK_x_ODTa = util.I_gaussian_beam_3D(E9c.lambda_lw, r_x_range_ODTs, l_x_range_ODTs, w0_ODTa_hori, w0_ODTa_vert, theta = 0)
norm_V_uK_x_ODTb = util.I_gaussian_beam_3D(E9c.lambda_lw, r_x_range_ODTs, l_x_range_ODTs, w0_ODTb_hori, w0_ODTb_vert, theta = 0)
norm_V_uK_y_ODTa = util.I_gaussian_beam_3D(E9c.lambda_lw, r_y_range_ODTs, l_y_range_ODTs, w0_ODTa_hori, w0_ODTa_vert, theta = 0)
norm_V_uK_y_ODTb = util.I_gaussian_beam_3D(E9c.lambda_lw, r_y_range_ODTs, l_y_range_ODTs, w0_ODTb_hori, w0_ODTb_vert, theta = 0)
norm_V_uK_z_ODTa = util.I_gaussian_beam_3D(E9c.lambda_lw, z_range_trap, 0., w0_ODTa_hori, w0_ODTa_vert, theta = np.pi / 2)
norm_V_uK_z_ODTb = util.I_gaussian_beam_3D(E9c.lambda_lw, z_range_trap, 0., w0_ODTb_hori, w0_ODTb_vert, theta = np.pi / 2)
V_uK_Rb_x_ODTa = V_uK_ODTa_Rb * norm_V_uK_x_ODTa
V_uK_Rb_x_ODTb = V_uK_ODTb_Rb * norm_V_uK_x_ODTb
V_uK_Rb_y_ODTa = V_uK_ODTa_Rb * norm_V_uK_y_ODTa
V_uK_Rb_y_ODTb = V_uK_ODTb_Rb * norm_V_uK_y_ODTb
V_uK_Rb_z_ODTa = V_uK_ODTa_Rb * norm_V_uK_z_ODTa
V_uK_Rb_z_ODTb = V_uK_ODTb_Rb * norm_V_uK_z_ODTb
V_uK_K_x_ODTa = V_uK_ODTa_K * norm_V_uK_x_ODTa
V_uK_K_x_ODTb = V_uK_ODTb_K * norm_V_uK_x_ODTb
V_uK_K_y_ODTa = V_uK_ODTa_K * norm_V_uK_y_ODTa
V_uK_K_y_ODTb = V_uK_ODTb_K * norm_V_uK_y_ODTb
V_uK_K_z_ODTa = V_uK_ODTa_K * norm_V_uK_z_ODTa
V_uK_K_z_ODTb = V_uK_ODTb_K * norm_V_uK_z_ODTb
V_uK_Rb_x = V_uK_Rb_x_ODTa + V_uK_Rb_x_ODTb
V_uK_Rb_y = V_uK_Rb_y_ODTa + V_uK_Rb_y_ODTb
V_uK_Rb_z = V_uK_Rb_z_ODTa + V_uK_Rb_z_ODTb
V_uK_K_x = V_uK_K_x_ODTa + V_uK_K_x_ODTb
V_uK_K_y = V_uK_K_y_ODTa + V_uK_K_y_ODTb
V_uK_K_z = V_uK_K_z_ODTa + V_uK_K_z_ODTb

# y-axis
l_y_range_ODTs = y_range_trap * np.sin(theta_ODTa)
r_y_range_ODTs = y_range_trap * np.cos(theta_ODTa)

# z-axis
r_z_range_ODTs = z_range_trap


## Gravitational potential

In [9]:
# No input

## Plot

In [ ]:
# Plot
fig_y0z0cut, ax_y0z0cut = plt.subplots(1, 1)
x_range_um = x_range_trap * 1e6
ax_y0z0cut.plot(x_range_um, V_uK_Rb_tot_xz[z0_index], color = Rb_plot_color, label = "Rb")
ax_y0z0cut.plot(x_range_um, V_uK_Rb_B_xz[z0_index], ls = "--", color = Rb_plot_color, alpha = 0.5)
ax_y0z0cut.plot(x_near_Vmin_y0z0_Rb * 1e6, V_uK_Rb_ho_y0z0, ls = ":", color = Rb_plot_color)
ax_y0z0cut.plot(x_range_um, V_uK_K_tot_xz[z0_index], color = K_plot_color, label = "K")
ax_y0z0cut.plot(x_range_um, V_uK_K_B_xz[z0_index], ls = "--", color = K_plot_color, alpha = 0.5)
ax_y0z0cut.plot(x_near_Vmin_y0z0_K * 1e6, V_uK_K_ho_y0z0, ls = ":", color = K_plot_color)
ax_y0z0cut.set_ylim([0, 200])
ax_y0z0cut.set_xlabel("x [um]")
ax_y0z0cut.set_ylabel("V [uK]")
ax_y0z0cut.legend()
ax_y0z0cut.set_title("y = 0, z = 0")
fig_y0z0cut.suptitle(fig_title_trap_params)

## Initialize trap geometry